<a href="https://colab.research.google.com/github/talibaisi/CoLab/blob/master/engine_v3_0_0_1_IoT2023_Classifier_multiAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scikeras catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import shap
# ✅ Disable HuggingFace transformer model detection in SHAP
shap.utils.transformers.is_transformers_lm = lambda x: False

In [4]:
# Assistantxxxx
# Assistant
# First, install the missing package
#!pip install tensorflow==2.10.0 keras==2.10.0 scikeras --upgrade
#!pip install catboost

# Then your imports can work correctly
from sklearn.linear_model import SGDClassifier  # Corrected import - SGDClassifier is in linear_model, not model_selection
from sklearn.datasets import make_classification
from sklearn.preprocessing import scale
import os
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer
import joblib
from joblib import dump, load
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import skew, kurtosis
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.ensemble import StackingClassifier,BaggingClassifier, GradientBoostingClassifier
from sklearn.base import clone

from imblearn.over_sampling import SMOTE, ADASYN

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Use the updated import for KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import tensorflow as tf
from sklearn.model_selection import GridSearchCV

import datetime

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import IsolationForest,StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier  # This will work after installation
from sklearn.metrics import matthews_corrcoef
import torch.nn.functional as F
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel


#General hyper parameters
K_FOLD_SPLITS=5
EPOCHS = 5 #100
RESAMPLE_ALGORITHM = 'SMOTE' #'ADASYN' #USE 'NONE' in case no sampling
RESAMPLING_RATIO = 1.0
LIME_FORCE_PLOT_SAMPLE_SLICE = 100

best_results = {
'Naive Bayes': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Logistic Regression': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Decision Tree': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'K-Nearest Neighbor': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Support Vector Machine': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Random Forest': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'XGBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'AdaBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'CatBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Bagging': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Gradient Boosting': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'CNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'RNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'LSTM Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'GRU Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
#'GNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Stacking(Logistic Regression)': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Stacking(Decision Tree)': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None}
}
#################### multi label (attack_type)
best_results_attack_types= {
'Naive Bayes': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Logistic Regression': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Decision Tree': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'K-Nearest Neighbor': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Support Vector Machine': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Random Forest': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'XGBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'AdaBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'CatBoost': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Bagging': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Gradient Boosting': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'CNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'RNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'LSTM Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'GRU Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
#'GNN Classifier': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"fpr":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Stacking(Logistic Regression)': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None},
'Stacking(Decision Tree)': {"accuracy": 0, "fold": -1,"f1":-1,"recall":-1,"precision_weighted":-1,"precision":-1,"roc_auc":-1,"confusion_mat":None,"MCC":-1,"g_mean": -1, "estimator": None, "X_train": None, "X_test": None, "y_train": None, "y_test": None,"file_path":None}
}
# Rest of your code remains the same

In [5]:
#==================================================
def is_stacking_tree_model(model_name, estimator):
    if 'Stacking' in model_name and isinstance(estimator, StackingClassifier):
        if isinstance(estimator.final_estimator_, (DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier)):
            return True
    return False

def is_stacking_non_tree_model(model_name, estimator):
    if 'Stacking' in model_name and isinstance(estimator, StackingClassifier):
        if isinstance(estimator.final_estimator_, (LogisticRegression, SVC, KNeighborsClassifier)):
            return True
    return False

def logExecTime(strMethodName, start_time, end_time):
    # Calculate the execution time in seconds
    execution_time_seconds = end_time - start_time

    # Convert the execution time to minutes
    execution_time_minutes = execution_time_seconds / 60

    # Print the execution time in minutes
    print(str(strMethodName) + " (Execution time) : {:.2f} minutes".format(execution_time_minutes))

    print('')
    print('-------------------')
    print(str(strMethodName) + " (Execution time) : {:.2f} minutes".format(execution_time_minutes))
    print('-------------------')
    print('')
def calculate_g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    g_mean = np.sqrt(sensitivity * specificity)
    return g_mean

#Function to calculate and log skewness and kurtosis
def log_skewness_kurtosis(data, feature_names):

    print("")
    print("***********************************************************************")
    print("log_skewness_kurtosis starts")
    print("")


    skewness = skew(data, axis=0)
    kurt = kurtosis(data, axis=0)
    for i, feature in enumerate(feature_names):
        print(f"Feature: {feature} - Skewness: {skewness[i]}, Kurtosis: {kurt[i]}")

    print("log_skewness_kurtosis ends")
    print("***********************************************************************")

# Define a function to split a floating-point value and return both parts
def split_float(value):
    try:
        float_value = float(value)
        before_dot = int(float_value)
        after_dot = float_value - before_dot
        return before_dot, after_dot
    except ValueError:
        return None, None


#Recursive To Vigesemal
def tovigisemal(decimalstring):
    dec = int(decimalstring)
    x = (dec % 20)
    digits = "0123456789ABCDEFGHIJ"
    rest = dec / 20
    if (rest == 0):
        return digits[x]
    return tovigisemal(rest) + digits[x]

def sendJobFinishAlertMail(sFrom="ta0002@gmail.com",sTo="ta0002@gmail.com",sSubject="Job Finished",sMessage="Your job is finished!"):
    # Email configuration
    sender_email = sFrom
    receiver_email = sTo
    password = "ywdorcecnqwqmsiz"

    # Create a MIMEText object
    message = MIMEText(sMessage)

    # Set the email subject
    message["Subject"] = sSubject

    # Connect to the SMTP server
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, password)

        # Send the email
        server.sendmail(sender_email, receiver_email, message.as_string())

        # Close the server connection
        server.quit()
        print("Email sent successfully!")

    except Exception as e:
        print(f"Email could not be sent. Error: {str(e)}")

    return

def create_folder_if_not_exists(folder_path):
  # Check if the folder exists
  if not os.path.exists(folder_path):
      # Create the folder if it doesn't exist
      os.makedirs(folder_path)
      print(f"Folder '{folder_path}' created.")
      print('*******************************************************')
      print('')
      print(f"Folder '{folder_path}' created.")
      print('')
  else:
      print(f"Folder '{folder_path}' already exists.")
      print('**************')
      print('')
      print(f"Folder '{folder_path}' already exists.")

  print('*******************************************************')


In [6]:

sMsg = 'This ML (CIC_IoT2023 ,Re-Sampling(Upsampling-SMOTE-Yeo-Johnson-transformation for multi attack, RATIOs 100%), also test Unseen 20% data with columns original order'
# Example usage:
print(sMsg)
print('************************************************************')
print('')

folder_path = './best_accuracy_models_multi_attack'
create_folder_if_not_exists(folder_path)
attack_type_encoding_folder_path = './attack_type_encoding'
create_folder_if_not_exists(attack_type_encoding_folder_path)

This ML (CIC_IoT2023 ,Re-Sampling(Upsampling-SMOTE-Yeo-Johnson-transformation for multi attack, RATIOs 100%), also test Unseen 20% data with columns original order
************************************************************

Folder './best_accuracy_models_multi_attack' created.
*******************************************************

Folder './best_accuracy_models_multi_attack' created.

*******************************************************
Folder './attack_type_encoding' created.
*******************************************************

Folder './attack_type_encoding' created.

*******************************************************


In [7]:
def ResampleDataFrameForMultiLabel(df_all, classCol="Label",commentOnDataframe=""):
        # Assuming 'label' is the column indicating benign (0) or attack (1)
        # Split the data into features (X) and labels (y)
        X = df_all.drop(classCol, axis=1)
        y = df_all[classCol]

        # Resample the imbalanced dataset using SMOTE or ADASYN
        # Choose one of the following resampling techniques
        # oversampler = SMOTE(sampling_strategy=0.5)  # SMOTE
        print('')
        print('****************************************************************')
        print('RESAMPLE ('+commentOnDataframe+') using '+RESAMPLE_ALGORITHM+' algorithm with ratio '+str(RESAMPLING_RATIO))
        print('****************************************************************')
        print('')

        print('y.unique() = '+str(y.unique()))
        print('y.value_counts() = '+str(y.value_counts()))
        print('')

        if RESAMPLE_ALGORITHM == 'SMOTE':
            oversampler = SMOTE(sampling_strategy=RESAMPLING_RATIO)  # SMOTE

            # Option 1: Resample all minority classes to match the majority
            oversampler = SMOTE(sampling_strategy='not majority')  # or 'auto' (same effect)

            # Option 2: Resample all classes to the same count
            #oversampler = SMOTE(sampling_strategy='not majority')  # good default

            # Option 3: Define manually if needed
            # oversampler = SMOTE(sampling_strategy={0: 1000, 1: 1000, 2: 1000, 3: 1000})
        else:
            oversampler = ADASYN(sampling_strategy=RESAMPLING_RATIO)  # ADASYN


        X_resampled, y_resampled = oversampler.fit_resample(X, y)

        # Create a DataFrame from the resampled data
        df_all = pd.DataFrame(data=X_resampled, columns=X.columns)
        df_all[classCol] = y_resampled
        return df_all

In [8]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
import pickle

# Function to train and evaluate the model
def train_and_evaluate_multi_attacks(estimator, name, X_train, y_train, X_test, y_test,k,best_results_attack_types):
    print(" ")
    print(" ---------------------------------------------- ")
    print("-----------------train_and_evaluate_multi_attacks()---------- starts")
    print(" ")
    print("This evaluation result is from Machine Learning model train_and_evaluate_multi_attacks()")

    # Set options for NumPy printing
    np.set_printoptions(precision=5, suppress=True)

    # Fit the estimator and predict
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)

    mcc = matthews_corrcoef(y_test, y_pred)

    g_mean = np.sqrt(np.prod(recall_score(y_test, y_pred, average=None)))
    #g_mean = calculate_g_mean(y_test, y_pred)

    print("******************************")
    print(f"Matthews correlation coefficient (MCC) for {name}: {mcc}")
    print(f"G-mean for {name}: {g_mean}")
    print("******************************")


    # Generate the confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    precision_weighted = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')


    # Example:
    classes = np.unique(y_test)
    y_test_bin = label_binarize(y_test, classes=classes)
    y_pred_prob = estimator.predict_proba(X_test)

    roc_auc = roc_auc_score(y_test_bin, y_pred_prob, average='macro', multi_class='ovr')

    #fpr = fp / (fp + tn)

    # Log classification report
    # Print classification report
    encoder_path = "/content/drive/MyDrive/Colab Notebooks/CICIoT2023/attack_type_encoding/attack_type_label_encoder.pkl"

    with open(encoder_path, 'rb') as f:
        le = pickle.load(f)

    print(le.classes_)  # e.g., ['Benign', 'DDoS', 'DoS', ...]

    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    #print('Classification Report:')
    #print(classification_report(y_test, y_pred))

    # Print confusion matrix
    print('Confusion Matrix:')
    print(cm)

    print("\n===== Evaluation Metrics for {} =====".format(name))
    print("Here are specific Evaluation Metrics values:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")
    print(f"ROC AUC: {roc_auc:.3f}")
    #print(f"False Positive Rate (FPR): {fpr:.3f}")

    if accuracy > best_results_attack_types[name]["accuracy"]:
        best_results_attack_types[name] = {
            "accuracy": accuracy,
            "fold": k,
            "f1":f1,
            "recall":recall,
            "precision_weighted":precision_weighted,
            "precision":precision,
            "roc_auc":roc_auc,
            "confusion_mat":cm, # cm.ravel()=tn, fp, fn, tp
            "MCC":mcc,
            "g_mean": g_mean,
            "estimator": estimator,
            "X_train": X_train,
            "X_test": X_test,
            "y_train": y_train,
            "y_test": y_test
        }


    print(" ")
    print("******************************** ")
    print(" ")
    print("-----------------train_and_evaluate_multi_attacks()---------- ends")
    return best_results_attack_types

In [9]:
####################### for Traditional ML training using CPU
def readSeqStructDataFromTable_TraditionalML_attackTypes_Training(NetSource,best_results_attack_types):

    try:
        print('***********************************************************')
        print('********readSeqStructDataFromTable_TraditionalML_attackTypes_Training() starts *****')


        CSVFileName = '/content/drive/MyDrive/Colab Notebooks/CICIoT2023/CICIoT2023_StructProp_multi_attacks.csv'

        # get all records
        sColLst = ['MolecularWeight', 'Aromaticity', 'InstabilityIndex', 'IsoelectricPoint',
                   'AlphaHelix', 'ReducedCysteines', 'DisulfideBridges', 'Gravy', 'BetaTurn', 'BetaStrand', 'IsBad']
        df_ModelDataSource = pd.read_csv(CSVFileName)

        # Filter only rows where NetSource == 'CIC_IoT_2023' (training set)
        df_ModelDataSource = df_ModelDataSource[df_ModelDataSource['NetSource'] == NetSource]
        df_ModelDataSource = df_ModelDataSource.drop(columns=['Id','Sequence', 'NetSource','Created_at'], errors='ignore')


        print('readSeqStructDataFromTable_TraditionalML_Training: # of Records = ' + str(df_ModelDataSource.shape) +
                     ' - for NetSource = ' + str(NetSource))

        ##################### Here do Resampling #############################
        print('***********************************************************')
        print('********ResampleDataFrameForMultiLabel() starts *****')
        print('***********************************************************')
        print('')

        start_time = time.time()
        df_ModelDataSource = ResampleDataFrameForMultiLabel(df_ModelDataSource, classCol="IsBad",commentOnDataframe="Training Data of Structural Properties for Multi attack types")
        end_time = time.time()
        logExecTime("ResampleDataFrameForMultiLabel", start_time, end_time)

        print('')
        print('***********************************************************')
        print('********ResampleDataFrameForMultiLabel() ends *****')
        print('***********************************************************')
        print('')

        ##################### END Resampling #################################

        print('***********************************************************')
        print('')
        print('After Resample, Total Training rows count (Benign & Attack) = '+str(len(df_ModelDataSource)))
        print('')
        import pickle

        # Load the saved label encoder
        encoder_path = "/content/drive/MyDrive/Colab Notebooks/CICIoT2023/attack_type_encoding/attack_type_label_encoder.pkl"
        with open(encoder_path, "rb") as f:
            label_encoder = pickle.load(f)

        # Decode labels and show counts with original attack names
        df_ModelDataSource['attack_type_decoded'] = label_encoder.inverse_transform(df_ModelDataSource['IsBad'])
        print('\nAfter Resample - Training rows count by Attack Type:')
        print(df_ModelDataSource['attack_type_decoded'].value_counts())
        df_ModelDataSource = df_ModelDataSource.drop(columns=['attack_type_decoded'], errors='ignore')
        print('***********************************************************')


        # Separate features and labels
        X = df_ModelDataSource.loc[:, ~df_ModelDataSource.columns.isin(['IsBad'])]
        y = df_ModelDataSource['IsBad']

        # Split data into training and testing sets
        stratified_splitter = StratifiedShuffleSplit(n_splits=K_FOLD_SPLITS, test_size=0.2, random_state=42)
        print("")
        print('----------------------Data preprocessing (Yeo-Johnson Transformation) starts')

        # Apply Yeo-Johnson transformation
        pt = PowerTransformer(method='yeo-johnson')
        X_transformed = pt.fit_transform(X)
        #X_test_transformed = pt.transform(X_test)

        print('----------------------Data preprocessing (Yeo-Johnson Transformation) ends')
        print("")

        print('***********************************************************')
        print('Calculating Skewness & Kurtosis values for Training data(X_train) after applying yeo-johnson transformation')
        print('')

        log_skewness_kurtosis(X_transformed, X.columns)

        # Optionally, perform feature scaling
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_transformed)

        # Initialize traditional ML classifiers
        nMLIterations = 100 #standard=1000
        n_MLestimators=10 #standard=50
        num_classes = len(np.unique(y))  # use your encoded y

        classifiers = {
            'Naive Bayes': GaussianNB(),
            'Logistic Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=nMLIterations),
            'Decision Tree': DecisionTreeClassifier(random_state=42),
            'K-Nearest Neighbor': KNeighborsClassifier(),
            'Support Vector Machine': SVC(probability=True, decision_function_shape='ovr'),
            'Random Forest': RandomForestClassifier(random_state=42),
            'XGBoost': XGBClassifier(objective='multi:softprob', num_class=num_classes, use_label_encoder=False, eval_metric='mlogloss'),
            'AdaBoost': AdaBoostClassifier(algorithm='SAMME', random_state=42),
            'CatBoost': CatBoostClassifier(iterations=nMLIterations, learning_rate=0.1, depth=6, verbose=0),
            'Bagging': BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=n_MLestimators, random_state=42),
            'Gradient Boosting': GradientBoostingClassifier(random_state=42)
        }

        k=0
        for train_index, test_index in stratified_splitter.split(X_scaled, y):
            X_train_scaled, X_test_scaled = X_scaled[train_index], X_scaled[test_index]
            y_train, y_test = y[train_index], y[test_index]

            print("******************************** ")
            print(" ")
            k+=1
            print("*********** K fold No = "+str(k))
            print(" ")

            # Train and evaluate each classifier
            for name, classifier in classifiers.items():
                print(f"Training and evaluating classifier: {name} ...")
                best_results_attack_types = train_and_evaluate_multi_attacks(classifier,name, X_train_scaled, y_train, X_test_scaled, y_test,k,best_results_attack_types)

                ############################# print performance

            # Create stacking classifiers
            num_classes = len(np.unique(y))  # Use the encoded label array

            stacking_classifiers = [
                ('rf', RandomForestClassifier(random_state=42)),
                ('xgb', XGBClassifier(objective='multi:softprob', num_class=num_classes, use_label_encoder=False, eval_metric='mlogloss')),
                ('ada', AdaBoostClassifier(algorithm='SAMME', random_state=42)),
                ('cat', CatBoostClassifier(iterations=nMLIterations, learning_rate=0.1, depth=6, verbose=0))
            ]

            # Define meta-models
            meta_models = {
                'Stacking(Logistic Regression)': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=nMLIterations),
                'Stacking(Decision Tree)': DecisionTreeClassifier(random_state=42)
            }

                # Train and evaluate stacking classifiers
            for meta_name, meta_model in meta_models.items():
                stacking_clf = StackingClassifier(
                    estimators=stacking_classifiers,
                    final_estimator=meta_model,
                    passthrough=True,
                    cv=K_FOLD_SPLITS,
                    n_jobs=-1  # Optional: enables parallelism
                )
                print(f"Training and evaluating stacking classifier with meta-model: {meta_name} ...")

                best_results_attack_types = train_and_evaluate_multi_attacks(stacking_clf, meta_name, X_train_scaled, y_train, X_test_scaled, y_test,k,best_results_attack_types)

        print('***********************************************************')
        print('readSeqStructDataFromTable_TraditionalML_Training ends')
        print('***********************************************************')

    except OSError as err:
        print("OS Error: " + str(err))

        raise RuntimeError from err
    except BaseException as err:
        print("Unexpected exception " + str(err))

        raise RuntimeError from err
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise RuntimeError
    return best_results_attack_types

In [1]:
NetSource='CIC_IoT_2023_multi_attack'
best_results_attack_types = readSeqStructDataFromTable_TraditionalML_attackTypes_Training(NetSource,best_results_attack_types)

NameError: name 'readSeqStructDataFromTable_TraditionalML_attackTypes_Training' is not defined